In [ ]:
# !pip install -e /files/shared/ap/jupyter-notebook/pkgs/objFuncs
# !pip install -e /files/shared/ap/jupyter-notebook/pkgs/pyBO

 # please re-start kernel if these packages are installed for the first time

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import time
import datetime
import concurrent
from phantasy import caget

In [ ]:
import objFuncs
from objFuncs import maximize_FC
from objFuncs.util import get_MEBT_objective_goal_from_BPMoverview
from objFuncs.util import plot_obj_history
from pyBO import pyBO

In [ ]:
budget = 96
n_init = 48

# check objFuncs machineIO, source and beam  

In [ ]:
objFuncs._global_machineIO._test = False
objFuncs._global_machineIO._fetch_data_time_span = 2.05
# objFuncs._global_machineIO.view()

In [ ]:
SCS = caget("ACS_DIAG:DEST:ACTIVE_ION_SOURCE")
ion = caget("FE_ISRC"+str(SCS)+":BEAM:ELMT_BOOK")
Q = caget("FE_ISRC"+str(SCS)+":BEAM:Q_BOOK")
A = caget("FE_ISRC"+str(SCS)+":BEAM:A_BOOK")
# AQ = caget("FE_ISRC2:BEAM:MOVRQ_BOOK")
AQ = A/Q
ion = str(A)+ion+str(Q)
print('SCS'+str(SCS), ion, 'A/Q=',AQ)

In [ ]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

In [ ]:
now0str = datetime.datetime.now().strftime('%Y%m%d_%H%M')
fname = now0str+'['+ion+']'+nb_name.strip('.ipynb')
fname

# preprare objective

### Decision parameters and range

##### couple two V-Edipoles

In [ ]:
# decision_CSETs=[
#                  'FE_LEBT:PSD1_D0833:V_CSET', # V-Edipole to be coupled with PSD2_D0936
#                  'FE_LEBT:PSC2_D0821:I_CSET', 'FE_LEBT:PSC2_D0821:I_CSET',  # one COR pair in U-LEBT  
#                  'FE_LEBT:PSC2_D0929:I_CSET',  # one HCOR
#                  'FE_LEBT:PSC2_D0948:I_CSET', 'FE_LEBT:PSC1_D0948:I_CSET',  # one COR pair in L-LEBT
#                  'FE_LEBT:PSC2_D0979:I_CSET', 'FE_LEBT:PSC1_D0979:I_CSET',  # one COR pair in L-LEBT
#                  ]

# PSD_ref = 0.5*(caget("FE_LEBT:PSD1_D0833:V_CSET")+caget("FE_LEBT:PSD1_D0936:V_CSET"))
# err = 1e-3  # 90 deg *1e-3 = 1.5 mrad   (10 meter * sin(1.5mrad) = 15 mm)

# decision_min = [PSD_ref*(1-err),   
#                 -0.1*AQ,-0.1*AQ,
#                 -0.4*AQ,
#                 -0.4*AQ,-0.4*AQ,
#                 -0.4*AQ,-0.4*AQ,]

# decision_max = [PSD_ref*(1+err),
#                  0.1*AQ, 0.1*AQ,
#                  0.4*AQ,
#                  0.4*AQ, 0.4*AQ,
#                  0.4*AQ, 0.4*AQ,]

# decision_couplings = {'FE_LEBT:PSD1_D0833:V_CSET': {'FE_LEBT:PSD2_D0833:V_CSET':-1, 
#                                                     'FE_LEBT:PSD1_D0936:V_CSET': 1,
#                                                     'FE_LEBT:PSD2_D0936:V_CSET':-1 }
#                      } 

##### independent two V-Edipoles

In [ ]:
decision_CSETs=[
                 'FE_LEBT:PSD1_D0833:V_CSET', 'FE_LEBT:PSD1_D0936:V_CSET',  # two V-Edipole
                 'FE_LEBT:PSC2_D0821:I_CSET' ,'FE_LEBT:PSC2_D0929:I_CSET',  # two HCOR
                 'FE_LEBT:PSC2_D0948:I_CSET', 'FE_LEBT:PSC1_D0948:I_CSET',  # one COR pair in L-LEBT
                 'FE_LEBT:PSC2_D0979:I_CSET', 'FE_LEBT:PSC1_D0979:I_CSET',  # one COR pair in L-LEBT
                 ]

PSD1_D833_ref, PSD1_D936_ref = caget("FE_LEBT:PSD1_D0833:V_CSET"),caget("FE_LEBT:PSD1_D0936:V_CSET")
err = 1e-3  # 90 deg *1e-3 = 1.5 mrad   (10 meter * sin(1.5mrad) = 15 mm)

decision_min = [PSD1_D833_ref*(1-err), PSD1_D936_ref*(1-err),   
                -0.1*AQ,-0.1*AQ,
                -0.4*AQ,-0.4*AQ,
                -0.4*AQ,-0.4*AQ ]

decision_max = [PSD1_D833_ref*(1+err), PSD1_D936_ref*(1+err),
                 0.1*AQ, 0.1*AQ,
                 0.4*AQ, 0.4*AQ,
                 0.4*AQ, 0.4*AQ ]

decision_couplings = {'FE_LEBT:PSD1_D0833:V_CSET': {'FE_LEBT:PSD2_D0833:V_CSET':-1},
                      'FE_LEBT:PSD1_D0936:V_CSET': {'FE_LEBT:PSD2_D0936:V_CSET':-1}
                     } 

In [ ]:
# FC814 = 22.2
FC998 = 11

### read from BPM snapshot file

In [ ]:
# BPM_snapshot_fname = '20230313_1008_48Ca10_197p0MeVu_to_target_20W_pulsed.bpm'
# objective_goal = get_MEBT_objective_goal_from_BPMoverview(BPM_snapshot_fname)
# objective_goal['FE_MEBT:FC_D1102:PKAVG_RD'] = {'more than': 0.8*FC998}
# objective_goal

#### manual defintion of objectives

In [ ]:
objective_goal = {
    'FE_MEBT:BPM_D1056:XPOS_RD': 0,
     'FE_MEBT:BPM_D1056:YPOS_RD': 0,
     'FE_MEBT:BPM_D1056:PHASE_RD': 78.72239428974802,
     'FE_MEBT:BPM_D1056:MAG_RD': {'more than': None},
     'FE_MEBT:BPM_D1072:XPOS_RD': 0,
     'FE_MEBT:BPM_D1072:YPOS_RD': 0,
     'FE_MEBT:BPM_D1072:PHASE_RD': -23.09912673431342,
     'FE_MEBT:BPM_D1072:MAG_RD': {'more than': None},
     'FE_MEBT:BPM_D1094:XPOS_RD': 0,
     'FE_MEBT:BPM_D1094:YPOS_RD': 0,
     'FE_MEBT:BPM_D1094:PHASE_RD': -15.344825463580433,
     'FE_MEBT:BPM_D1094:MAG_RD': {'more than': None},
     'FE_MEBT:BCM_D1055:AVGPK_RD/FE_LEBT:BCM_D0989:AVGPK_RD': {'more than': 1.0},
     'FE_MEBT:FC_D1102:PKAVG_RD': {'more than': 0.8*FC998} #0.8*FC814
     }

In [ ]:
objective_norm = { 
    'FE_MEBT:BPM_D1056:XPOS_RD' : 1.,     
    'FE_MEBT:BPM_D1056:YPOS_RD' : 1.,     
    'FE_MEBT:BPM_D1056:PHASE_RD': 1., 
    'FE_MEBT:BPM_D1056:MAG_RD'  : 0, 
    'FE_MEBT:BPM_D1072:XPOS_RD' : 1.,     
    'FE_MEBT:BPM_D1072:YPOS_RD' : 1.,     
    'FE_MEBT:BPM_D1072:PHASE_RD': 2., 
    'FE_MEBT:BPM_D1072:MAG_RD'  : 0, 
    'FE_MEBT:BPM_D1094:XPOS_RD' : 1.,     
    'FE_MEBT:BPM_D1094:YPOS_RD' : 1.,     
    'FE_MEBT:BPM_D1094:PHASE_RD': 3.,
    'FE_MEBT:BPM_D1094:MAG_RD'  : 0,
    'FE_MEBT:BCM_D1055:AVGPK_RD/FE_LEBT:BCM_D0989:AVGPK_RD': 0.05,
    'FE_MEBT:FC_D1102:PKAVG_RD': 0.1*FC998,#0.1*FC814,
    }

In [ ]:
objective_weight = { 
    'FE_MEBT:BPM_D1056:XPOS_RD' : 0.5,     
    'FE_MEBT:BPM_D1056:YPOS_RD' : 0.5,     
    'FE_MEBT:BPM_D1056:PHASE_RD': 1., 
    'FE_MEBT:BPM_D1056:MAG_RD'  : 0., 
    'FE_MEBT:BPM_D1072:XPOS_RD' : 0.,     
    'FE_MEBT:BPM_D1072:YPOS_RD' : 0.,     
    'FE_MEBT:BPM_D1072:PHASE_RD': 0., 
    'FE_MEBT:BPM_D1072:MAG_RD'  : 0., 
    'FE_MEBT:BPM_D1094:XPOS_RD' : 0.0,     
    'FE_MEBT:BPM_D1094:YPOS_RD' : 0.0,     
    'FE_MEBT:BPM_D1094:PHASE_RD': 0.0,
    'FE_MEBT:BPM_D1094:MAG_RD'  : 0,
    'FE_MEBT:BCM_D1055:AVGPK_RD/FE_LEBT:BCM_D0989:AVGPK_RD': 3,
    'FE_MEBT:FC_D1102:PKAVG_RD': 2.,
    }

### regularization

In [ ]:
reg_weight_ratio = 0.05

reg_goal = {key:0.5*(val_max+val_min) if 'PSD' in key else 0 for key,val_min,val_max in zip(decision_CSETs,decision_min,decision_max)}
reg_norm = {key:0.5*(val_max-val_min) for key,val_min,val_max in zip(decision_CSETs,decision_min,decision_max)}
if len(decision_couplings) > 2:
    reg_weight = {key:0.05 for key in decision_CSETs}
else:  # account for coupled two V-Edipole
    reg_weight = {key:0.1 if 'PSD' in key else 0.05 for key in decision_CSETs}   
    
ratio = np.sum(list(reg_weight.values()))/np.sum(list(objective_weight.values()))
reg_weight = {k:reg_weight_ratio*v/ratio for k,v in reg_weight.items()}

In [ ]:
objective_goal.update(reg_goal)
objective_norm.update(reg_norm)
objective_weight.update(reg_weight)

In [ ]:
# pd.DataFrame((objective_goal,objective_norm,objective_weight),index=['goal','norm','weight']).T

### construct objectives class

In [ ]:
obj = maximize_FC.maximize_FC1102(
    decision_CSETs = decision_CSETs,
    decision_couplings = decision_couplings,
    decision_min = decision_min,
    decision_max = decision_max,
    objective_goal = objective_goal,
    objective_weight = objective_weight,
    objective_norm = objective_norm,
    apply_bilog = True,
    history_logging_fname = fname,
    history_logging_frequency = np.inf,
)

### Prepare plot callbacks

In [ ]:
# things to plot
BPM_XYs = [key for key in obj.objective_weight.keys() if ':XPOS_' in key or ':YPOS_' in key]
BPM_PHASEs = [key for key in obj.objective_weight.keys() if ':PHASE_' in key]
BPM_MAGs = [key for key in obj.objective_weight.keys() if ':MAG_' in key]
FCs = [key for key in obj.objective_weight.keys() if ':FC_' in key]
BCMs = ['FE_LEBT:BCM_D0989:AVGPK_RD','FE_MEBT:BCM_D1055:AVGPK_RD']
BCMratio = ['FE_MEBT:BCM_D1055:AVGPK_RD/FE_LEBT:BCM_D0989:AVGPK_RD']

In [ ]:
# define what to plot
plot_CSETs = plot_obj_history(
                obj.history['decision_CSETs'],
                keys=[ [key for key in obj.history['decision_CSETs']['names'] if 'PSD1' in key],
                       [key for key in obj.history['decision_CSETs']['names'] if 'PSD2' in key],
                       [key for key in obj.history['decision_CSETs']['names'] if 'PSC' in key],
                     ],
                title = 'decision_CSETs',
                inline = True,
                )
plot_RDs = plot_obj_history(
                obj.history['objective_RDs'],
                keys=[ BPM_XYs, BPM_PHASEs, BCMs+FCs],
                title = 'objective_RDs',
                inline = True,
                )
plot_objs = plot_obj_history(
            obj.history['objectives'],
            keys=[ BPM_XYs, BPM_PHASEs, BCMratio, FCs],
            title = 'objectives',
            inline = True,
            add_y_data = obj.history['objectives']['total'],
            add_y_label = 'total obj'
            )
callbacks = [plot_CSETs,plot_RDs,plot_objs]

if 'FE_LEBT:PSD1_D0833:V_CSET' in decision_CSETs:
    plot_couplings = plot_obj_history(
                       obj.history['coupled_decision_CSETs'],
                       keys=[ [key for key in obj.history['coupled_decision_CSETs']['names'] if 'PSD1' in key],
                              [key for key in obj.history['coupled_decision_CSETs']['names'] if 'PSD2' in key],
                       ], 
                       title = 'decision_couplings',
                       inline = True,
                       )
    callbacks.insert(1,plot_couplings)


# evaluate objective and plot
def obj_callbacks(x):
    return obj(x,callbacks=callbacks)

# Run BO

### run BO until budget exhaust

In [ ]:
# run optimizer
bo, X_pending, Y_pending_future = pyBO.runBO(
                                    obj_callbacks,  
                                    bounds=obj.decision_bounds,
                                    n_init=n_init,
                                    budget = budget,
                                    batch_size=1,
#                                     timeout = 2+objFuncs._global_machineIO._fetch_data_time_span,
                                    path="./log/",
                                    tag=fname+'_pyBO_history',
                                    write_log = False)


# exploit model for a few iteration
acquisition_func_args = {'beta':0.01}
X_pending, Y_pending_future= bo.loop( 
                                n_loop=2,  # number of additional optimization interation
                                func_obj = obj_callbacks,
                                bounds = obj.decision_bounds,
                                acquisition_func_args = acquisition_func_args,
                                X_pending = X_pending, 
                                Y_pending_future = Y_pending_future,
                                batch_size = 1,
                                write_log = False,
                                )

for f in callbacks:
    f.close()

In [ ]:
now1 = datetime.datetime.now()
now1str = str(now1)[:str(now1).rfind(':')].replace(' ','_').replace(':','').replace('-','')
time_delta = now1 - now0
print("time took:",str(time_delta.total_seconds()/60)[:4],'min')

### Fine Tune near current best

In [ ]:
# local optimization near the best_solution
bounds_diff = obj.decision_bounds[:,1] - obj.decision_bounds[:,0]
for i in range(3):
    x_best,y_best = bo.best_sofar()
    bounds_best = np.array(list(zip(x_best-0.08*bounds_diff, x_best+0.08*bounds_diff)))
    acquisition_func_args = {'beta':1}

    X_pending, Y_pending_future= bo.loop( 
                                    n_loop=3,  # number of additional optimization interation
                                    func_obj = obj_callbacks,
                                    bounds = bounds_best,
                                    acquisition_func_args = acquisition_func_args,
                                    X_pending = X_pending, 
                                    Y_pending_future = Y_pending_future,
                                    batch_size = 1,
                                    write_log = False,
                                    )
for f in callbacks:
    f.close()

In [ ]:
ax = bo.plot_obj_history()
ax.vlines(n_init,*ax.get_ylim(),color='k')

In [ ]:
# set to best solution 
x_best,y_best_old = bo.best_sofar()
y_best_new = obj(x_best)
print(x_best,y_best_old[0],y_best_new)   # check if best solution objective value is consistent

In [ ]:
df = pd.DataFrame(obj.history['objective_RDs']['values'][-1], index=obj.history['objective_RDs']['names'])
df

In [ ]:
(df.loc["FE_MEBT:BCM_D1055:AVGPK_RD"]/df.loc["FE_LEBT:BCM_D0989:AVGPK_RD"]).values[0]

In [ ]:
obj.write_log(fname=os.path.join('./log',fname))

# change objective goal/norm/weights. 
##### Note that if an objective weight were zero previously, that objective cannot be added.
objFunc does not read nor record data corresponding to the zero weight

In [ ]:
# objective_weight = { 
#     'FE_MEBT:BPM_D1056:XPOS_RD' : 0.5,     
#     'FE_MEBT:BPM_D1056:YPOS_RD' : 0.5,     
#     'FE_MEBT:BPM_D1056:PHASE_RD': 1., 
#     'FE_MEBT:BPM_D1056:MAG_RD'  : 0., 
#     'FE_MEBT:BPM_D1072:XPOS_RD' : 0.,     
#     'FE_MEBT:BPM_D1072:YPOS_RD' : 0.,     
#     'FE_MEBT:BPM_D1072:PHASE_RD': 0., 
#     'FE_MEBT:BPM_D1072:MAG_RD'  : 0., 
#     'FE_MEBT:BPM_D1094:XPOS_RD' : 0.0,     
#     'FE_MEBT:BPM_D1094:YPOS_RD' : 0.0,     
#     'FE_MEBT:BPM_D1094:PHASE_RD': 0.0,
#     'FE_MEBT:BPM_D1094:MAG_RD'  : 0,
#     'FE_MEBT:BCM_D1055:AVGPK_RD/FE_LEBT:BCM_D0989:AVGPK_RD': 3,
#     'FE_MEBT:FC_D1102:PKAVG_RD': 3.,
#     }

# obj.update_objective(objective_weight=objective_weight)

# x = np.array(obj.history['decision_CSETs']['values'][:-1])
# y = np.array(obj.history['objectives']['total'])[:-1,None]
# bo.update_model(x=x,y=y)

# x_best,y_best = bo.best_sofar()
# print(x_best, y_best)

In [ ]:
# acquisition_func_args = {'beta':9.}
# X_pending, Y_pending_future= bo.loop( 
#                             n_loop=10,  # number of additional optimization interation
#                             func_obj = obj_callbacks,
#                             bounds = obj.decision_bounds,
#                             acquisition_func_args = acquisition_func_args,
#                             X_pending = X_pending, 
#                             Y_pending_future = Y_pending_future,
#                             batch_size = 1,
#                             write_log = False,
#                             )
# for f in callbacks:
#     f.close()

In [ ]:
# # local optimization near the best_solution
# x_best,y_best = bo.best_sofar()
# bounds_best = np.array([(x-0.1*AQ,x+0.1*AQ) for x in x_best])   
# acquisition_func_args = {'beta':0.4}

# X_pending, Y_pending_future= bo.loop( 
#                                 n_loop=4,  # number of additional optimization interation
#                                 func_obj = obj_callbacks,
#                                 bounds = bounds_best,
#                                 acquisition_func_args = acquisition_func_args,
#                                 X_pending = X_pending, 
#                                 Y_pending_future = Y_pending_future,
#                                 batch_size = 1,
#                                 write_log = False,
#                                 )
# for f in callbacks:
#     f.close()

In [ ]:
# ax = bo.plot_obj_history()
# ax.vlines(n_init,*ax.get_ylim(),color='k')

In [ ]:
# # set to best solution 
# x_best,y_best_old = bo.best_sofar()
# y_best_new = obj(x_best)
# print(x_best,y_best_old[0],y_best_new)   # check if best solution objective value is consistent

In [ ]:
# obj.write_log(fname=os.path.join('./log',fname))

# Visualize Surrogate model

In [ ]:
# fixed_values_for_each_dim = {2:x_best[2],3:x_best[3]}  # fix values to visualize high dim surrogate model
fixed_values_for_each_dim = None                         # do not fix values but project maximum. Can take long time to plot
batch_size = 1

In [ ]:
# plot surrogate mean model of the last epoch. Can take long time for decision dim >= 4

t0 = time.monotonic()
from math import ceil
nplot = int(0.5*len(obj.decision_CSETs))
nrow = ceil(0.5*nplot)
fig,ax = plt.subplots(nrow,2,figsize=(8,3*nrow))
for i in range(nrow):
    for j in range(2):
        n = 2*i+j
        if nrow>1:
            ax_ = ax[i,j]
        else:
            ax_ = ax[j]
        if n >= nplot:
            ax_.set_visible(False)
            break
        bo.plot_model_2D_projection(project_maximum=True,
                                    dim_xaxis = 2*n,
                                    dim_yaxis = 2*n+1,
                                    bounds = obj.decision_bounds,
                                    grid_ponits_each_dim = 16,
                                    fixed_values_for_each_dim=fixed_values_for_each_dim,
                                            fig=fig,ax=ax_);
        ax_.scatter(obj.x0[2*n],obj.x0[2*n+1],color='k',label='initial')
        ax_.scatter(x_best[2*n],x_best[2*n+1],color='r',label='optimum')
        ax_.set_xlabel(obj.decision_CSETs[2*n  ])
        ax_.set_ylabel(obj.decision_CSETs[2*n+1])
        ax_.legend()
fig.tight_layout()

    
print('time took: ',time.monotonic()-t0)